# Getting Started

In [46]:
%reload_ext autoreload
%autoreload 2
from recsys.MovieLens import MovieLens
from surprise import SVD

In [47]:
def create_anti_test_set(user_id, trainset):
    fill = trainset.global_mean
    anti_testset = []
    inner_uid = trainset.to_inner_uid(str(user_id))
    
    user_items = set([j for (j, _) in trainset.ur[inner_uid]])
    anti_testset += [(trainset.to_raw_uid(inner_uid), trainset.to_raw_iid(i), fill) for
                             i in trainset.all_items() if
                             i not in user_items]
    return anti_testset

In [48]:
# Pick an arbitrary test subject
test_subject = 85

lens, data, _ = MovieLens.load()

user_ratings = lens.get_user_ratings(test_subject)
loved = []
hated = []
for ratings in user_ratings:
    if (float(ratings[1]) > 4.0):
        loved.append(ratings)
    if (float(ratings[1]) < 3.0):
        hated.append(ratings)

loved.sort(key=lambda x: x[1], reverse=True)
hated.sort(key=lambda x: x[1])

print(f"User {test_subject} loved these movies:")
for ratings in loved[:10]:
    print(lens.get_movie_name(ratings[0]), ratings[1])
print("\n...and hated these movies:")
for ratings in hated[:10]:
    print(lens.get_movie_name(ratings[0]), ratings[1])

User 85 loved these movies:
Jumanji (1995) 5.0
GoldenEye (1995) 5.0
Braveheart (1995) 5.0
Jerky Boys, The (1995) 5.0
LÃ©on: The Professional (a.k.a. The Professional) (LÃ©on) (1994) 5.0
Pulp Fiction (1994) 5.0
Stargate (1994) 5.0
Shawshank Redemption, The (1994) 5.0
Star Trek: Generations (1994) 5.0
Clear and Present Danger (1994) 5.0

...and hated these movies:
Postman, The (Postino, Il) (1994) 1.0
Casper (1995) 1.0
Mighty Morphin Power Rangers: The Movie (1995) 1.0
Prophecy, The (1995) 1.0
Heavenly Creatures (1994) 1.0
Little Women (1994) 1.0
Nell (1994) 1.0
Poison Ivy II (1996) 1.0
Tank Girl (1995) 1.0
While You Were Sleeping (1995) 1.0


In [49]:
# Building recommendation model
train_set = data.build_full_trainset()

algo = SVD(verbose=False)
algo.fit(train_set)

In [50]:
# Generate recommendations
test_set = create_anti_test_set(test_subject, train_set)
predictions = algo.test(test_set)

recommendations = []

data = lens.load_movielens_data()

print ("We recommend:\n")
for user_id, movie_id, true_rating, predicted_rating, _ in predictions:
    recommendations.append((int(movie_id), predicted_rating))

recommendations.sort(key=lambda x: x[1], reverse=True)

for ratings in recommendations[:10]:
    print(lens.get_movie_name(ratings[0]), f"{ratings[1]:.1f}")

We recommend:

Godfather, The (1972) 4.6
Happiness (1998) 4.4
Star Wars: Episode V - The Empire Strikes Back (1980) 4.4
Producers, The (1968) 4.4
Wallace & Gromit: The Best of Aardman Animation (1996) 4.4
Reservoir Dogs (1992) 4.4
It Happened One Night (1934) 4.4
Crimes and Misdemeanors (1989) 4.4
Departed, The (2006) 4.4
Cube (1997) 4.3
